This try to install the required libraries in your Jupyter server. Run once.

In [24]:
import sys
!{sys.executable} -m pip install pyserial
!{sys.executable} -m pip install pymeasure

     |████████████████████████████████| 301 kB 4.7 MB/s            
     |████████████████████████████████| 11.3 MB 6.3 MB/s            
     |████████████████████████████████| 17.0 MB 6.7 MB/s            
     |████████████████████████████████| 992 kB 6.8 MB/s            
     |████████████████████████████████| 503 kB 6.4 MB/s            


Import the library and select the python visa backend

In [43]:
from pymeasure.instruments.agilent import Agilent34410A

In [73]:
dmm = Agilent34410A('ASRL/dev/cu.usbserial-8020')


In [74]:
#keysight.write("*rst; status:preset; *cls")
dmm.reset()
print(dmm.id)
voltage = keysight.query("READ?")
print("voltage: ", voltage)

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.